# 프로젝트
> 처음에는 여러명의 작가와 시인의 데이터를 학습을 진행하였을때 
생성되는 문장에 특별한 특징들이 거의 없었다.

그래서 특정 인물 한명을 지정하여서 학습을 진행해주었다.


##모듈 불러오기

In [ ]:
import tensorflow as tf
import numpy as np
import os
from pprint import pprint
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

##데이터(텍스트) 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path ='/content/drive/MyDrive/bootcamp/project04/한용운시인.txt'
text = open(file_path,'rb').read().decode(encoding='utf-8')

In [ ]:
print(text[:500])

님의 침묵


님은 갔습니다. 아아 사랑하는 나의 님은 갔습니다.

푸른 산빛을 깨치고, 단풍나무 숲을 향하여 난, 작은 길을 걸어서 차마 떨치고 갔습니다.

황금의 꽃같이 굳고 빛나던 옛 맹서는 차디찬 티끌이 되어서, 한숨의 미풍에 날아갔습니다.

날카로운 첫 「키스」의 추억은 나의 운명의 지침을 돌려놓고, 뒷걸음쳐서 사라졌습니다.

나는 향기로운 님의 말소리에 귀먹고, 꽃다운 님의 얼굴에 눈멀었습니다.

사랑도 사람의 일이라, 만날 때에 미리 떠날 것을 염려하고 경계하지 아니한 것은 아니지만, 이별은 뜻밖의 일이 되고 놀란 가슴은 새로운 슬픔에 터집니다.

그러나 이별을 쓸데없는 눈물의 원천을 만들고 마는 것은 스스로 사랑을 깨뜨리는 것인 줄 아는 까닭에, 걷잡을 수 없는 슬픔의 힘을 옮겨서 새 희망의 정수박이에 들이부었습니다.

우리는 만날 때에 떠날 것을 염려하는 것과 같이, 떠날 때에 다시 만날 것을 믿습니다.

아아 님은 갔지마는 나는 님


## 텍스트 전처리

* 불러온 텍스트 파일(courpus)의 전체 길이

In [ ]:
len(text)

10778

* 고유 캐릭터의 수

In [ ]:
print(sorted(list(set(text))))

['\n', '\r', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '<', '>', 'A', 'D', 'E', 'G', 'I', 'N', 'O', 'R', 'S', 'T', '[', ']', '「', '」', '一', '三', '世', '亢', '人', '光', '冥', '出', '動', '化', '反', '合', '和', '國', '垂', '報', '塔', '境', '天', '奮', '宮', '尼', '岸', '平', '弱', '形', '從', '微', '恨', '恩', '悲', '情', '想', '慈', '我', '摩', '操', '收', '斷', '明', '春', '昧', '晶', '服', '杜', '桐', '梧', '橋', '權', '殿', '民', '氣', '水', '永', '江', '法', '流', '海', '淨', '滅', '漂', '烏', '無', '狂', '玉', '珊', '珠', '瑚', '璽', '白', '直', '神', '福', '秋', '空', '窮', '笑', '籍', '紅', '絲', '美', '群', '義', '者', '聖', '肉', '菩', '萬', '薩', '詩', '貞', '質', '遠', '酒', '間', '靑', '面', '顔', '骨', '鵑', '鵲', '黃', '가', '각', '간', '갈', '감', '갑', '갔', '강', '갖', '같', '갚', '개', '거', '건', '걷', '걸', '검', '것', '게', '겠', '겨', '격', '견', '결', '겸', '경', '계', '고', '곡', '골', '곱', '곳', '공', '과', '광', '괴', '교', '구', '국', '군', '굳', '굴', '궁', '권', '귀', '그', '근', '글', '금', '급', '기', '길', '김', '깃', '깊', '까', '깨', '꽃', '꾸', '꿈', '끄', '끊', '끌', '끗', '끝', '끼', '나', '난', '날', '낡', '남

In [ ]:
# 좀더 보기 편하게 
# for i in sorted(list(set(text))):
#     print(i)

In [ ]:
before_chars = sorted(list(set(text)))
len(before_chars)

661

정규표현식을 사용해서 텍스트를 정리해준다.

In [ ]:
import re
def del_special(text):
    #한글, 띄어쓰기, 줄바꿈 이외는 제외
    after_text = re.sub(r"[^가-힣\n ]","",text)
    return after_text

함수적용 이후 변경된 텍스트 확인

In [ ]:
after_text = del_special(text)
print(after_text[:500])

님의 침묵


님은 갔습니다 아아 사랑하는 나의 님은 갔습니다

푸른 산빛을 깨치고 단풍나무 숲을 향하여 난 작은 길을 걸어서 차마 떨치고 갔습니다

황금의 꽃같이 굳고 빛나던 옛 맹서는 차디찬 티끌이 되어서 한숨의 미풍에 날아갔습니다

날카로운 첫 키스의 추억은 나의 운명의 지침을 돌려놓고 뒷걸음쳐서 사라졌습니다

나는 향기로운 님의 말소리에 귀먹고 꽃다운 님의 얼굴에 눈멀었습니다

사랑도 사람의 일이라 만날 때에 미리 떠날 것을 염려하고 경계하지 아니한 것은 아니지만 이별은 뜻밖의 일이 되고 놀란 가슴은 새로운 슬픔에 터집니다

그러나 이별을 쓸데없는 눈물의 원천을 만들고 마는 것은 스스로 사랑을 깨뜨리는 것인 줄 아는 까닭에 걷잡을 수 없는 슬픔의 힘을 옮겨서 새 희망의 정수박이에 들이부었습니다

우리는 만날 때에 떠날 것을 염려하는 것과 같이 떠날 때에 다시 만날 것을 믿습니다

아아 님은 갔지마는 나는 님을 보내지 아니하였습니다

제 곡조를 못 이기는 사랑의 노래는 님의 침


In [ ]:
vocab = sorted(set(list(after_text)))

In [ ]:
print("처리된 문자갯수 :",len(before_chars) - len(vocab))
print("최종 캐릭터수 :",len(vocab))

처리된 문자갯수 : 130
최종 캐릭터수 : 531


char to index와 index to char를 만들어준다.


In [ ]:
char2idx = {value: count for count, value in enumerate(vocab)}
pprint(char2idx)

{'\n': 0,
 ' ': 1,
 '가': 2,
 '각': 3,
 '간': 4,
 '갈': 5,
 '감': 6,
 '갑': 7,
 '갔': 8,
 '강': 9,
 '갖': 10,
 '같': 11,
 '갚': 12,
 '개': 13,
 '거': 14,
 '건': 15,
 '걷': 16,
 '걸': 17,
 '검': 18,
 '것': 19,
 '게': 20,
 '겠': 21,
 '겨': 22,
 '격': 23,
 '견': 24,
 '결': 25,
 '겸': 26,
 '경': 27,
 '계': 28,
 '고': 29,
 '곡': 30,
 '골': 31,
 '곱': 32,
 '곳': 33,
 '공': 34,
 '과': 35,
 '광': 36,
 '괴': 37,
 '교': 38,
 '구': 39,
 '국': 40,
 '군': 41,
 '굳': 42,
 '굴': 43,
 '궁': 44,
 '권': 45,
 '귀': 46,
 '그': 47,
 '근': 48,
 '글': 49,
 '금': 50,
 '급': 51,
 '기': 52,
 '길': 53,
 '김': 54,
 '깃': 55,
 '깊': 56,
 '까': 57,
 '깨': 58,
 '꽃': 59,
 '꾸': 60,
 '꿈': 61,
 '끄': 62,
 '끊': 63,
 '끌': 64,
 '끗': 65,
 '끝': 66,
 '끼': 67,
 '나': 68,
 '난': 69,
 '날': 70,
 '낡': 71,
 '남': 72,
 '납': 73,
 '낮': 74,
 '내': 75,
 '냅': 76,
 '냐': 77,
 '너': 78,
 '널': 79,
 '넘': 80,
 '네': 81,
 '녀': 82,
 '녁': 83,
 '년': 84,
 '념': 85,
 '노': 86,
 '녹': 87,
 '놀': 88,
 '놉': 89,
 '높': 90,
 '놓': 91,
 '누': 92,
 '눈': 93,
 '뉘': 94,
 '느': 95,
 '는': 96,
 '늘': 97,
 '능': 98,
 '늦': 99,
 '니': 100

In [ ]:
idx2char = np.array(vocab)
pprint(idx2char)

array(['\n', ' ', '가', '각', '간', '갈', '감', '갑', '갔', '강', '갖', '같', '갚',
       '개', '거', '건', '걷', '걸', '검', '것', '게', '겠', '겨', '격', '견', '결',
       '겸', '경', '계', '고', '곡', '골', '곱', '곳', '공', '과', '광', '괴', '교',
       '구', '국', '군', '굳', '굴', '궁', '권', '귀', '그', '근', '글', '금', '급',
       '기', '길', '김', '깃', '깊', '까', '깨', '꽃', '꾸', '꿈', '끄', '끊', '끌',
       '끗', '끝', '끼', '나', '난', '날', '낡', '남', '납', '낮', '내', '냅', '냐',
       '너', '널', '넘', '네', '녀', '녁', '년', '념', '노', '녹', '놀', '놉', '높',
       '놓', '누', '눈', '뉘', '느', '는', '늘', '능', '늦', '니', '닐', '님', '닙',
       '다', '단', '달', '닭', '답', '당', '대', '더', '덕', '던', '덤', '데', '도',
       '돌', '돕', '동', '되', '될', '됩', '두', '둘', '뒤', '뒷', '드', '득', '든',
       '듣', '들', '듯', '등', '디', '딜', '따', '땅', '때', '떠', '떨', '떻', '또',
       '뚜', '뚤', '뜨', '뜻', '띄', '라', '락', '란', '람', '랍', '랐', '랑', '래',
       '랴', '러', '런', '럼', '럽', '렀', '렁', '렇', '레', '려', '련', '로', '록',
       '롭', '루', '룩', '룻', '류', '률', '르', '른', '를', '름', '릅', '

텍스트 변환

tmi)
tai : text as index

In [ ]:
tai = np.array([char2idx[c] for c in after_text])

In [ ]:
len(tai)

9466

In [ ]:
#int로 변환된 값 확인
print(tai)

[102 384   1 ... 390 100 104]


In [ ]:
filter_size = 4
#단풍나무 숲을 향하여 이 10개
shuffle_buffer = 10
batch_size = 4

In [ ]:
def windowed_dataset(series, window_size, shuffle_buffer, batch_size):
    series = tf.expand_dims(series, -1) #차원 변경
    ds = tf.data.Dataset.from_tensor_slices(series)#tf.dataset으로 type변경
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1)) #차원 펼쳐주기
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda x: (x[:-1], x[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
train = windowed_dataset(np.array(tai),filter_size,shuffle_buffer,batch_size)

In [ ]:
vocab_size = len(vocab)
vocab_size

531

In [ ]:
embedding_dim = 256
rnn_units = 1024

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size
                    ,embedding_dim
                    ,batch_input_shape=[batch_size,None]))
model.add(LSTM(rnn_units
               ,return_sequences=True # many to many 
               ,stateful=True
               ,recurrent_initializer='glorot_uniform'))
model.add(Dense(vocab_size))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (4, None, 256)            135936    
                                                                 
 lstm_5 (LSTM)               (4, None, 1024)           5246976   
                                                                 
 dense_5 (Dense)             (4, None, 531)            544275    
                                                                 
Total params: 5,927,187
Trainable params: 5,927,187
Non-trainable params: 0
_________________________________________________________________


checkpoint 설정

In [ ]:
checkpoint_path = './models/checkpt.ckpt'

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True, 
    save_best_only=True,
    monitor='loss', 
    verbose=1, 
)

loss 설정

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(train, 
          epochs=100, 
          steps_per_epoch=100, 
          callbacks=[checkpoint_callback])

Epoch 1/100
100/100 [==============================] - ETA: 0s - loss: 1.2309 - accuracy: 0.7182
Epoch 00001: loss did not improve from 1.04025
100/100 [==============================] - 42s 417ms/step - loss: 1.2309 - accuracy: 0.7182
Epoch 2/100
100/100 [==============================] - ETA: 0s - loss: 1.0241 - accuracy: 0.7685
Epoch 00002: loss improved from 1.04025 to 1.02405, saving model to ./models/checkpt.ckpt
100/100 [==============================] - 42s 417ms/step - loss: 1.0241 - accuracy: 0.7685
Epoch 3/100
100/100 [==============================] - ETA: 0s - loss: 1.1323 - accuracy: 0.7230
Epoch 00003: loss did not improve from 1.02405
100/100 [==============================] - 41s 414ms/step - loss: 1.1323 - accuracy: 0.7230
Epoch 4/100
100/100 [==============================] - ETA: 0s - loss: 0.9971 - accuracy: 0.7530
Epoch 00004: loss improved from 1.02405 to 0.99706, saving model to ./models/checkpt.ckpt
100/100 [==============================] - 42s 417ms/step - lo

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size
                    ,embedding_dim
                    ,batch_input_shape=[1,None]))
model.add(LSTM(rnn_units
               ,return_sequences=True # many to many 
               ,stateful=True
               ,recurrent_initializer='glorot_uniform'))
model.add(Dense(vocab_size))

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (1, None, 256)            135936    
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_6 (Dense)             (1, None, 531)            544275    
                                                                 
Total params: 5,927,187
Trainable params: 5,927,187
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string, num):
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

    # 생성할 문자의 수
    num_generate = num

    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 결과를 저장할 빈 문자열
    text_generated = []

    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 온도가 높으면 더 의외의 텍스트가 됩니다.
    # 최적의 세팅을 찾기 위한 실험
    temperature = 1.0

    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)

        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 예측된 단어를 다음 입력으로 모델에 전달
        # 이전 은닉 상태와 함께
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"지금부터 ",num=500))

지금부터 터춘은 차마 그리는 까닭입니다

손이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 작은 별만 없는 아닙니다

다른 사람들은 나의 건강만을 사랑하지마는 당신은 나의 꿈은 눈물이 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 맑은 바람이 되어서 여름날에 더위를 못 이기어 낮잠을 자거든 나의 건강만을 얻겠습니다

체체만법이 되어서 당신의 머리 위에 지키고 꿈이라면

끝없이 당신의 머리 위에 지키고 마음이 이라면

사랑의 속박이 꿈이라면




당신이 맑에서 당신의 머리 위에 놓입니까



꿈이라면

사한 것은 아닙니다

하금을 맑은 가까워지고 마음이 꿈이라면

무심의 광명도 꿈입니다

웃음과 눈물이 꿈이라면

사랑의 꿈에서 꿈이 되어서 바술에 하뿐이 난다



고 술에 사람은 작은 별이 되어서 당신의 머리 위에 지키고 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

웃음과 눈물이 꿈이라면

무심의 광명도 꿈입니다

웃음과


In [ ]:
print(generate_text(model, start_string=u"꿈이라면 ",num=800))

꿈이라면 나무 그날은 더 미소만을 까닭입니다

출세의 해탈도 꿈입니다


당신이 맑은 새벽에 나무 그날 줄이라면

사랑의 꿈에서 불멸을 얻겠습니다

웃음과 눈물이 꿈이라면

사랑의 광명도 꿈입니다



거지의 거친 밭에 고 얼음 바다에는 어린 풀의 흔드는 누구의 입김만을 사랑하지마는 당신은 나의 꿈은 작은 별이 되어서 당신의 머리 위에 갔습니다

웃음과 눈물이 꿈이라면

무심의 광명도 꿈입니다

웃음과 눈물이 꿈이라면

무심의 광명도 꿈입니다

웃음과 눈물이 꿈이라면

무심의 머리 위에 심은 맑박이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 맑은 별이 되어서 당신의 머리 위에 지키고 연키기다리는 것은 아닙니다

까닭이 없는 것은 아닙니다

웃음과 눈물이 없는 것은 아닙니다

웃음과 눈물이 꿈이라면

만일이 꿈이라면

사랑의 속박이 꿈이라면

무심의 광명도 꿈입니다


나의 꿈은 맑은 바람이 되어서 당신의 머리 위에 집보다 나무에 푸른 이끼어 낮잠을 기루어서 걸음보다 멀어지더니 보이려다 말고 말려다 보인다


나의 꿈
다른 사람들은 나의 
꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

이 나의 꿈은 맑은 바람이 되어서 당신의 머리 위에 지키고 꿈이라면

사랑의 속박이 꿈이라면

사람은 멀어진다

일체만법이 꿈이라면

사랑의 광명도 꿈입니다







나의 꿈은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 허리워하는 것은 까닭이 없는 건강만을 사랑하에 마음은 가까워지고 마음이 꿈이라면

사랑의 속박이 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈


In [ ]:
print(generate_text(model, start_string=u"사랑",num=1000))

사랑이여 얼음 당신의 머리 위에 지키고 있겠습니다

고름의 소리여 


나의 꿈

사랑의 속박이 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

다른 사람들은 나의 죽음도 사랑하는 까닭입니다



당신이 맑은 새벽에 나무 그날 없는 것은 아닙니다

실버들 같은 입술 흰 이 탑 위에 지키고 있겠습니다



나의 꿈

당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 작은 별이 되어서 당신의 새탈도 꿈입니다

일체만법이 되어서 당신의 머리 위에 지키고 있겠습니다

사랑의 속박이 꿈이라면

사람은 맑은 맑은 별은 조각구름이 난다

다른 사람들은 나의 꿈은 맑은 바람이 되어서 당신의 머리 위에 지키고 말려다

보이는 듯한 내가 당신의 머리 산호가 되도록 도덕 그날 사이에서 산보할 때에 나의 꿈은 맑은 조각구름이 난다

사람들은 나의 꿈은 맑은 바다

사이 되어서 당신의 머리 위에 지키고 사랑이는 듯박이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 맑은 바람은 별이 되어서 당신의 머리 위에 지키어 낮잠을 얻겠습니다

당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 맑은 바람이 되어서 당신의 머리 위에 떠돌겠습니다



당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 맑은 바람이 되어서 당신의 머리 위에 려다리는 것은 까닭이 없는 것은 까닭이 없는 것은 아닙니다

웃음과 눈물이 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다


나의 건머리 위에 놓입니다




당신이 맑은 새벽에 나무 그날 사이에서 산보할 때에 나의 꿈은 작은 별이 되어서 당신의 머리 위에 미소만을 

내가 당신을 죽음도 꿈입니다

웃음과 눈물이 꿈이라면

사랑의 꿈에서 불멸을 얻겠습니다

시름의 광명도 꿈입니다

출세의 해탈도 꿈입니다


내가 당신의 머리 위에 지키고 있드 갈매기보다 작은 못 하리에 이고 싶어서 당신의 머리 위에 고 돌아왔습니다






꿈이라면

사랑의 속박이 꿈이라면

사랑의 속박이 꿈이라면

사이에서 산보할 때에 나의 꿈은 맑은 작은

한용운 문학의 특징은 불교사상과 독립사상이 탁월하게 예술적으로 결합된 데서 드러난다. <br>자유와 평등사상, 민족사상과 민중사상으로 요약되는 불교적 세계관과 독립사상은 한용운 문학의 뼈대이자 피와 살이라고 할 수 있기 때문이다.<br> 말하자면 그의 문학은 불교사상과 독립사상, 문학사상이 삼위일체를 이룬다는 점이 특징이라는 뜻이다.<br>
[출처: 한국민족문화대백과사전(한용운(韓龍雲))]